# 📊 People Analytics Dashboard

Este notebook interativo fornece análises avançadas e visualizações para os dados de avaliação de desempenho, permitindo insights detalhados sobre o desempenho individual e da equipe.

## 🔧 Configuração e Importações
Carregando as bibliotecas e configurações necessárias para análise avançada.

In [ ]:
# Importações necessárias
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from collections import defaultdict
from IPython.display import display, HTML, Markdown
import warnings

# Configurações para melhor exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

# Configuração de estilo para visualizações
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Cores para visualizações
COLORS = px.colors.qualitative.Plotly

# Estilo para HTML
HTML("""
<style>
    h1 { color: #2c3e50; }
    h2 { color: #34495e; border-bottom: 1px solid #95a5a6; padding-bottom: 5px; }
    h3 { color: #3498db; }
    .alert-success { background-color: #d4edda; color: #155724; padding: 15px; border-radius: 5px; }
    .alert-info { background-color: #d1ecf1; color: #0c5460; padding: 15px; border-radius: 5px; }
    .alert-warning { background-color: #fff3cd; color: #856404; padding: 15px; border-radius: 5px; }
    .alert-danger { background-color: #f8d7da; color: #721c24; padding: 15px; border-radius: 5px; }
    table { border-collapse: collapse; margin: 20px 0; }
    table th { background-color: #3498db; color: white; }
    table th, table td { padding: 10px; border: 1px solid #ddd; }
    table tr:nth-child(even) { background-color: #f2f2f2; }
</style>
""")

## 🛠️ Funções Utilitárias
Conjunto de funções para carregamento, processamento e análise de dados.

In [ ]:
def load_json_file(file_path):
    """Carrega um arquivo JSON com tratamento de erros
    
    Args:
        file_path (str): Caminho para o arquivo JSON
        
    Returns:
        tuple: (dados JSON ou None se erro, mensagem de erro ou None se sucesso)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f), None
    except json.JSONDecodeError as e:
        return None, f"Erro ao decodificar JSON: {str(e)}"
    except Exception as e:
        return None, f"Erro ao ler arquivo: {str(e)}"

def scan_directory(directory_path, pattern="*/*/resultado.json"):
    """Escaneia diretório procurando arquivos que seguem um padrão
    
    Args:
        directory_path (str): Caminho do diretório a ser escaneado
        pattern (str): Padrão glob a ser procurado
        
    Returns:
        list: Lista de caminhos de arquivos encontrados
    """
    path = Path(directory_path)
    files = list(path.glob(pattern))
    return files

def diagnose_json_file(file_path):
    """Diagnostica problemas em um arquivo JSON de avaliação
    
    Args:
        file_path (str): Caminho para o arquivo JSON
        
    Returns:
        dict: Dicionário com status da análise e problemas encontrados
    """
    data, error = load_json_file(file_path)
    if error:
        return {"file": str(file_path), "status": "error", "message": error}
    
    issues = []
    
    # Verificar estrutura básica
    if 'data' not in data:
        issues.append("Campo 'data' ausente")
    elif 'direcionadores' not in data['data']:
        issues.append("Campo 'direcionadores' ausente")
    elif not data['data']['direcionadores']:
        issues.append("Lista de direcionadores vazia")
    else:
        # Verificar cada direcionador e comportamento
        for i, direcionador in enumerate(data['data']['direcionadores']):
            if 'comportamentos' not in direcionador:
                issues.append(f"Direcionador {i+1} não tem campo 'comportamentos'")
                continue
                
            for j, comportamento in enumerate(direcionador.get('comportamentos', [])):
                if 'avaliacoes_grupo' not in comportamento:
                    issues.append(f"Comportamento {j+1} em Direcionador {i+1} não tem campo 'avaliacoes_grupo'")
                    continue
                    
                for k, avaliacao in enumerate(comportamento.get('avaliacoes_grupo', [])):
                    if 'frequencia_colaborador' not in avaliacao:
                        issues.append(f"Avaliação {k+1} em Comportamento {j+1} não tem campo 'frequencia_colaborador'")
                    elif not isinstance(avaliacao.get('frequencia_colaborador'), list):
                        issues.append(f"Campo 'frequencia_colaborador' não é uma lista em Avaliação {k+1}")
                        
                    if 'frequencia_grupo' not in avaliacao:
                        issues.append(f"Avaliação {k+1} em Comportamento {j+1} não tem campo 'frequencia_grupo'")
                    elif not isinstance(avaliacao.get('frequencia_grupo'), list):
                        issues.append(f"Campo 'frequencia_grupo' não é uma lista em Avaliação {k+1}")
    
    return {
        "file": str(file_path),
        "status": "ok" if not issues else "issues",
        "issues": issues,
        "data": data
    }

def extract_person_year(file_path):
    """Extrai nome da pessoa e ano a partir do caminho do arquivo
    
    Args:
        file_path (Path): Caminho do arquivo
        
    Returns:
        tuple: (nome da pessoa, ano)
    """
    parts = file_path.parts
    person_idx = len(parts) - 3
    year_idx = len(parts) - 2
    
    if person_idx >= 0 and year_idx >= 0:
        return parts[person_idx], parts[year_idx]
    return "Unknown", "Unknown"

def convert_evaluation_to_dataframe(eval_data, person, year):
    """Converte dados de avaliação para um DataFrame pandas
    
    Args:
        eval_data (dict): Dados de avaliação
        person (str): Nome da pessoa
        year (str): Ano da avaliação
        
    Returns:
        pandas.DataFrame: DataFrame com os dados estruturados
    """
    rows = []
    
    try:
        for direcionador in eval_data["data"]["direcionadores"]:
            for comportamento in direcionador.get("comportamentos", []):
                for avaliacao in comportamento.get("avaliacoes_grupo", []):
                    # Calcular scores médios
                    freq_colab = avaliacao.get("frequencia_colaborador", [])
                    freq_grupo = avaliacao.get("frequencia_grupo", [])
                    
                    if not freq_colab or not freq_grupo or sum(freq_colab) == 0 or sum(freq_grupo) == 0:
                        continue
                        
                    score_colab = sum(i * v for i, v in enumerate(freq_colab)) / sum(freq_colab)
                    score_grupo = sum(i * v for i, v in enumerate(freq_grupo)) / sum(freq_grupo)
                    
                    row = {
                        "pessoa": person,
                        "ano": year,
                        "direcionador": direcionador.get("direcionador", ""),
                        "comportamento": comportamento.get("comportamento", ""),
                        "avaliador": avaliacao.get("avaliador", ""),
                        "score_colaborador": score_colab,
                        "score_grupo": score_grupo,
                        "diferenca": score_colab - score_grupo,
                    }
                    rows.append(row)
    except Exception as e:
        print(f"Erro ao processar dados para {person}, {year}: {str(e)}")
    
    return pd.DataFrame(rows)

def load_all_evaluations(data_dir):
    """Carrega todas as avaliações de um diretório e retorna como DataFrame
    
    Args:
        data_dir (str): Caminho do diretório com os dados
        
    Returns:
        pandas.DataFrame: DataFrame consolidado com todos os dados
    """
    files = scan_directory(data_dir, "*/*/resultado.json")
    all_data = []
    
    for file_path in files:
        diagnosis = diagnose_json_file(file_path)
        if diagnosis["status"] == "error" or diagnosis["status"] == "issues":
            print(f"Problemas com arquivo {file_path}: {diagnosis.get('message') or diagnosis.get('issues')}")
            continue
            
        person, year = extract_person_year(file_path)
        df = convert_evaluation_to_dataframe(diagnosis["data"], person, year)
        all_data.append(df)
    
    if not all_data:
        return pd.DataFrame()
        
    return pd.concat(all_data, ignore_index=True)


## 📂 Carregamento dos Dados
Configure o caminho para o diretório de dados e carregue as avaliações.

In [ ]:
# Configurar caminho para diretório de dados
DATA_DIR = "test_data"  # Altere para o caminho dos seus dados reais

# Encontrar arquivos no padrão <pessoa>/<ano>/resultado.json
files = scan_directory(DATA_DIR, "*/*/resultado.json")
print(f"Encontrados {len(files)} arquivos de resultados:")
for f in files:
    print(f"  - {f}")

# Carregar todos os dados em um DataFrame
df_evaluations = load_all_evaluations(DATA_DIR)

if df_evaluations.empty:
    display(HTML('<div class="alert-danger">Nenhum dado de avaliação encontrado. Verifique o caminho especificado.</div>'))
else:
    display(HTML(f'<div class="alert-success">Carregados dados de {df_evaluations["pessoa"].nunique()} pessoas em {df_evaluations["ano"].nunique()} anos.</div>'))
    display(df_evaluations.head())

## 📊 Visão Geral dos Dados
Resumo estatístico e análise exploratória das avaliações.

In [ ]:
if not df_evaluations.empty:
    # Estatísticas gerais
    stats = {
        "Quantidade de pessoas": df_evaluations["pessoa"].nunique(),
        "Anos de avaliação": sorted(df_evaluations["ano"].unique()),
        "Direcionadores": df_evaluations["direcionador"].nunique(),
        "Comportamentos": df_evaluations["comportamento"].nunique(),
        "Tipos de avaliadores": df_evaluations["avaliador"].nunique(),
        "Score médio individual": df_evaluations["score_colaborador"].mean(),
        "Score médio grupo": df_evaluations["score_grupo"].mean(),
        "Diferença média": df_evaluations["diferenca"].mean()
    }
    
    # Exibir estatísticas em uma tabela HTML formatada
    html_stats = "<table>"
    html_stats += "<tr><th>Métrica</th><th>Valor</th></tr>"
    for key, value in stats.items():
        if isinstance(value, float):
            value_str = f"{value:.2f}"
        elif isinstance(value, list):
            value_str = ", ".join(str(v) for v in value)
        else:
            value_str = str(value)
        html_stats += f"<tr><td>{key}</td><td>{value_str}</td></tr>"
    html_stats += "</table>"
    
    display(HTML(html_stats))
    
    # Distribuição dos scores
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Distribuição de Scores Individual", "Distribuição de Diferenças"])
    
    fig.add_trace(
        go.Histogram(
            x=df_evaluations["score_colaborador"], 
            nbinsx=20, 
            marker_color=COLORS[0],
            name="Score Individual"
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=df_evaluations["diferenca"], 
            nbinsx=20, 
            marker_color=COLORS[1],
            name="Diferença vs Grupo"
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=400,
        title_text="Distribuição de Scores e Diferenças",
        showlegend=True
    )
    
    fig.show()

## 🧮 Análise por Pessoa
Comparação de desempenho entre diferentes pessoas avaliadas.

In [ ]:
if not df_evaluations.empty:
    # Scores médios por pessoa
    person_scores = df_evaluations.groupby("pessoa").agg(
        score_medio=("score_colaborador", "mean"),
        diferenca_media=("diferenca", "mean"),
        score_minimo=("score_colaborador", "min"),
        score_maximo=("score_colaborador", "max"),
        contagem=("score_colaborador", "count")
    ).reset_index()
    
    # Plotar gráfico de barras comparativo
    fig = px.bar(
        person_scores,
        x="pessoa",
        y="score_medio",
        error_y=person_scores["score_maximo"] - person_scores["score_medio"],
        error_y_minus=person_scores["score_medio"] - person_scores["score_minimo"],
        color="diferenca_media",
        size="contagem",
        hover_data=["score_minimo", "score_maximo", "contagem"],
        color_continuous_scale=px.colors.diverging.RdBu,
        color_continuous_midpoint=0,
        title="Score Médio por Pessoa (com barras de erro min-max)",
        labels={
            "pessoa": "Pessoa",
            "score_medio": "Score Médio",
            "diferenca_media": "Diferença vs Grupo",
            "score_minimo": "Score Mínimo",
            "score_maximo": "Score Máximo",
            "contagem": "Quantidade de Avaliações"
        }
    )
    
    fig.update_layout(
        height=500,
        xaxis_title="Pessoa",
        yaxis_title="Score Médio (0-4)",
        yaxis_range=[0, 4],
        coloraxis_colorbar=dict(title="Diferença vs Grupo")
    )
    
    fig.show()
    
    # Tabela detalhada por pessoa
    person_scores = person_scores.sort_values("score_medio", ascending=False)
    display(person_scores.style
           .format({"score_medio": "{:.2f}", "diferenca_media": "{:.2f}", 
                   "score_minimo": "{:.2f}", "score_maximo": "{:.2f}"})
           .background_gradient(cmap="RdYlGn", subset=["score_medio"])
           .background_gradient(cmap="RdBu", subset=["diferenca_media"])
          )

## ⚠️ Correção Importante na Interpretação dos Dados
Os vetores de frequência têm 6 posições com significados específicos.

In [ ]:
def convert_evaluation_to_dataframe_corrected(eval_data, person, year):
    """Converte dados de avaliação para DataFrame com interpretação correta dos vetores
    
    Args:
        eval_data (dict): Dados de avaliação
        person (str): Nome da pessoa
        year (str): Ano da avaliação
        
    Returns:
        pandas.DataFrame: DataFrame com os dados estruturados corretamente
    """
    rows = []
    
    # Rótulos para cada posição no vetor
    position_labels = ["não informado", "referência", "acima do esperado", 
                      "dentro do esperado", "abaixo do esperado", "muito abaixo do esperado"]
    
    # Valores para cálculo do score (invertido para que "acima do esperado" tenha maior valor)
    # Não informado recebe 0, referência é neutro, e os outros são por escala de valor
    position_values = [0, 2.5, 4, 3, 2, 1]
    
    try:
        for direcionador in eval_data["data"]["direcionadores"]:
            for comportamento in direcionador.get("comportamentos", []):
                for avaliacao in comportamento.get("avaliacoes_grupo", []):
                    freq_colab = avaliacao.get("frequencia_colaborador", [])
                    freq_grupo = avaliacao.get("frequencia_grupo", [])
                    
                    # Verificar se temos vetores válidos com 6 posições
                    if len(freq_colab) != 6 or len(freq_grupo) != 6 or sum(freq_colab) == 0 or sum(freq_grupo) == 0:
                        continue
                    
                    # Calcular scores ponderados pelos valores de cada posição
                    # Excluímos "não informado" do cálculo do total para não distorcer
                    total_colab = sum(freq_colab[1:]) # Excluindo posição 0 (não informado)
                    total_grupo = sum(freq_grupo[1:]) # Excluindo posição 0 (não informado)
                    
                    if total_colab == 0 or total_grupo == 0:
                        continue
                    
                    # Cálculo do score ponderado
                    score_colab = sum(v * position_values[i] for i, v in enumerate(freq_colab)) / total_colab
                    score_grupo = sum(v * position_values[i] for i, v in enumerate(freq_grupo)) / total_grupo
                    
                    # Criar distribuição percentual para visualização
                    dist_colab = [round(100 * v / sum(freq_colab), 1) if sum(freq_colab) > 0 else 0 for v in freq_colab]
                    dist_grupo = [round(100 * v / sum(freq_grupo), 1) if sum(freq_grupo) > 0 else 0 for v in freq_grupo]
                    
                    row = {
                        "pessoa": person,
                        "ano": year,
                        "direcionador": direcionador.get("direcionador", ""),
                        "comportamento": comportamento.get("comportamento", ""),
                        "avaliador": avaliacao.get("avaliador", ""),
                        "score_colaborador": score_colab,
                        "score_grupo": score_grupo,
                        "diferenca": score_colab - score_grupo,
                        "freq_colab_raw": freq_colab,
                        "freq_grupo_raw": freq_grupo,
                        "freq_colab_pct": dist_colab,
                        "freq_grupo_pct": dist_grupo
                    }
                    rows.append(row)
    except Exception as e:
        print(f"Erro ao processar dados para {person}, {year}: {str(e)}")
    
    return pd.DataFrame(rows)

def load_all_evaluations_corrected(data_dir):
    """Carrega todas as avaliações com interpretação correta dos vetores
    
    Args:
        data_dir (str): Caminho do diretório com os dados
        
    Returns:
        pandas.DataFrame: DataFrame consolidado com todos os dados
    """
    files = scan_directory(data_dir, "*/*/resultado.json")
    all_data = []
    
    for file_path in files:
        diagnosis = diagnose_json_file(file_path)
        if diagnosis["status"] == "error" or diagnosis["status"] == "issues":
            print(f"Problemas com arquivo {file_path}: {diagnosis.get('message') or diagnosis.get('issues')}")
            continue
            
        person, year = extract_person_year(file_path)
        df = convert_evaluation_to_dataframe_corrected(diagnosis["data"], person, year)
        all_data.append(df)
    
    if not all_data:
        return pd.DataFrame()
        
    return pd.concat(all_data, ignore_index=True)

# Carregar dados com método corrigido
position_labels = ["Não informado", "Referência", "Acima do esperado", "Dentro do esperado", "Abaixo do esperado", "Muito abaixo do esperado"]
df_corrected = load_all_evaluations_corrected(DATA_DIR)

if df_corrected.empty:
    display(HTML('<div class="alert-danger">Nenhum dado de avaliação encontrado com o método corrigido.</div>'))
else:
    display(HTML(f'<div class="alert-success">Dados carregados corretamente com nova interpretação dos vetores de frequência!</div>'))
    display(df_corrected.head())


## 📊 Visualização da Distribuição de Frequências
Análise da distribuição percentual em cada categoria de avaliação.

In [ ]:
if not df_corrected.empty:
    # Criar dataframe com médias das distribuições por pessoa
    people = df_corrected["pessoa"].unique()
    dist_data = []
    
    for person in people:
        person_data = df_corrected[df_corrected["pessoa"] == person]
        
        # Agregar porcentagens médias para cada categoria
        avg_colab = np.mean(np.stack(person_data["freq_colab_pct"].values), axis=0)
        
        for i, lbl in enumerate(position_labels):
            dist_data.append({
                "pessoa": person,
                "categoria": lbl,
                "porcentagem": avg_colab[i],
                "ordem": i
            })
    
    df_dist = pd.DataFrame(dist_data)
    
    # Visualizar distribuição por pessoa
    fig = px.bar(
        df_dist,
        x="pessoa", 
        y="porcentagem",
        color="categoria",
        barmode="stack",
        category_orders={"categoria": position_labels},
        color_discrete_sequence=px.colors.qualitative.Plotly,
        title="Distribuição de Avaliações por Pessoa",
        labels={
            "pessoa": "Pessoa",
            "porcentagem": "Porcentagem (%)",
            "categoria": "Categoria de Avaliação"
        }
    )
    
    fig.update_layout(
        height=500,
        yaxis_title="Porcentagem (%)",
        yaxis_range=[0, 100],
        legend_title_text="Categoria"
    )
    
    fig.show()
    
    # Criar um heatmap da distribuição média por pessoa
    pivot_data = df_dist.pivot_table(
        index="pessoa",
        columns="categoria",
        values="porcentagem",
        aggfunc="mean"
    )
    
    # Reordenar as colunas para a ordem correta das categorias
    pivot_data = pivot_data[position_labels]
    
    fig = px.imshow(
        pivot_data,
        text_auto=".1f",
        aspect="auto",
        color_continuous_scale="RdBu_r",
        title="Heatmap da Distribuição de Avaliações (%)",
        labels={
            "x": "Categoria",
            "y": "Pessoa",
            "color": "Porcentagem (%)"
        }
    )
    
    fig.update_layout(
        height=400,
        margin=dict(l=50, r=50, t=80, b=50)
    )
    
    fig.show()


## 📈 Scores Corrigidos
Análise dos scores recalculados com a interpretação correta dos vetores.

In [ ]:
if not df_corrected.empty:
    # Comparar scores com método correto vs anterior
    person_scores_corrected = df_corrected.groupby("pessoa").agg(
        score_medio=("score_colaborador", "mean"),
        diferenca_media=("diferenca", "mean"),
        score_minimo=("score_colaborador", "min"),
        score_maximo=("score_colaborador", "max"),
        contagem=("score_colaborador", "count")
    ).reset_index()
    
    # Radar chart por competência
    competency_scores = df_corrected.groupby(["pessoa", "direcionador"]).agg(
        score_medio=("score_colaborador", "mean")
    ).reset_index()
    
    # Criar um radar chart para cada pessoa
    fig = go.Figure()
    
    for person in competency_scores["pessoa"].unique():
        person_data = competency_scores[competency_scores["pessoa"] == person]
        
        fig.add_trace(go.Scatterpolar(
            r=person_data["score_medio"],
            theta=person_data["direcionador"],
            fill="toself",
            name=person
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 4]
            )
        ),
        title="Radar Chart de Competências por Pessoa",
        height=600,
        showlegend=True
    )
    
    fig.show()
    
    # Tabela de scores por direcionador e comportamento
    behavior_scores = df_corrected.groupby(["direcionador", "comportamento"]).agg(
        score_medio=("score_colaborador", "mean"),
        diferenca_media=("diferenca", "mean"),
        contagem=("score_colaborador", "count")
    ).reset_index().sort_values(["direcionador", "score_medio"], ascending=[True, False])
    
    display(behavior_scores.style
           .format({"score_medio": "{:.2f}", "diferenca_media": "{:.2f}"})
           .background_gradient(cmap="RdYlGn", subset=["score_medio"])
           .background_gradient(cmap="RdBu", subset=["diferenca_media"]))


## 📋 Recomendações e Ações
Principais insights e recomendações baseadas nos dados corrigidos.

In [ ]:
if not df_corrected.empty:
    # Identificar pontos fortes e oportunidades de melhoria
    html_insights = """
    <div class="alert-info">
    <h3>Insights Principais</h3>
    <p>Com base na interpretação correta dos vetores de frequência, identificamos:</p>
    <ul>
    """
    
    # Top competências
    top_comp = behavior_scores.sort_values("score_medio", ascending=False).iloc[0]
    html_insights += f"<li><strong>Competência mais forte:</strong> {top_comp["direcionador"]} - {top_comp["comportamento"]} ({top_comp["score_medio"]:.2f}/4.0)</li>"
    
    # Competências para desenvolvimento
    bottom_comp = behavior_scores.sort_values("score_medio").iloc[0]
    html_insights += f"<li><strong>Oportunidade de desenvolvimento:</strong> {bottom_comp["direcionador"]} - {bottom_comp["comportamento"]} ({bottom_comp["score_medio"]:.2f}/4.0)</li>"
    
    # Maiores gaps vs grupo
    if behavior_scores["diferenca_media"].min() < -0.5:
        gap_comp = behavior_scores.sort_values("diferenca_media").iloc[0]
        html_insights += f"<li><strong>Maior gap vs. grupo:</strong> {gap_comp["direcionador"]} - {gap_comp["comportamento"]} ({gap_comp["diferenca_media"]:.2f})</li>"
    
    html_insights += """
    </ul>
    </div>
    """
    
    display(HTML(html_insights))
    
    # Sugestões de ação por pessoa
    for person in df_corrected["pessoa"].unique():
        person_data = df_corrected[df_corrected["pessoa"] == person]
        person_behaviors = person_data.groupby(["direcionador", "comportamento"]).agg(
            score=("score_colaborador", "mean"),
            diferenca=("diferenca", "mean")
        ).reset_index()
        
        # Top strength
        top = person_behaviors.sort_values("score", ascending=False).iloc[0]
        # Area for improvement
        bottom = person_behaviors.sort_values("score").iloc[0]
        
        html_person = f"""
        <div class="alert-success">
        <h3>Sugestões para {person}</h3>
        <p><strong>Ponto forte a alavancar:</strong> {top["direcionador"]} - {top["comportamento"]} ({top["score"]:.2f}/4.0)</p>
        <p><strong>Área para desenvolvimento:</strong> {bottom["direcionador"]} - {bottom["comportamento"]} ({bottom["score"]:.2f}/4.0)</p>
        <p><strong>Recomendações:</strong></p>
        <ol>
        <li>Continuar demonstrando forte desempenho em {top["direcionador"]}, especialmente em {top["comportamento"]}.</li>
        <li>Desenvolver um plano de ação para melhorar {bottom["direcionador"]}, com foco em {bottom["comportamento"]}.</li>
        <li>Buscar feedback contínuo e estabelecer métricas claras para acompanhar o progresso.</li>
        </ol>
        </div>
        """
        
        display(HTML(html_person))
